In [1]:
import pandas as pd
import numpy as np

In [2]:
de = pd.read_csv('../differential_expression_analysis/output/edgr_exclude_one/intersect.tsv', sep='\t')
de = de.set_index('ensembl', drop=False)

mapp = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl', drop=False)

de_keep = pd.read_csv('../differential_expression_analysis/output/edgr_exclude_one/keep.intersect.txt', header=None)
de_keep = de_keep[0].to_list()
de_keep = mapp.loc[de_keep]['Description'].to_list()

outs = pd.read_csv('Supp_table_all_variants.SVs_included.csv')
outs = outs.set_index('gene', drop=False)

rel = pd.read_csv('../data/family_summaries.tsv', sep='\t')
rel = rel.set_index('subject', drop=False)
rel = rel.drop('SG011')

pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# format network df

In [3]:
net = pd.read_csv('all_gene_network_simple.tsv', sep='\t')

net.columns = ['startg', 'endg', 'distance',
       'num_connector_genes']

net = net.set_index(['startg', 'endg'], drop=False)

net = net.rename_axis((None, None))

# create df to hold output statistics

In [7]:
df = pd.DataFrame(index=pheno.subject)
df['avg_shortest_distance'] = 0.0

# outlier genes to wg variants

In [12]:
snv = pd.read_csv('../network_analysis/16p_all_snvs.csv', sep=',')

def str2float(s, default = 0):
    if s == '.':
        return default
    else:
        return float(s)

    
snv.CADD = snv.CADD.apply(lambda s: str2float(s))

lof = snv.Mut_type == 'lof'
cadd_10 = snv.CADD > 10
splice = snv.Mut_type == 'splice'

snv = snv[lof | cadd_10 | splice].copy()

In [13]:
cnv = pd.read_csv('../../cnv_parse/output/merged/16p_all_rare_cnvs.tsv', sep='\t')
cnv = cnv[cnv.gene_covered == 'encapsulated']

In [38]:
for sub in pheno.subject:
    print(sub)
    outlier_genes = outs[(outs['sample'] == sub) & (outs.outlier == 'X')].gene.to_list()
    
    snv_genes = snv[snv['Sample'] == sub]
    snv_genes = list(snv_genes['Gene.refGene'].unique())
    cnv_genes = cnv[cnv.subject == sub].gene.to_list()
    variant_genes = list(set(snv_genes + cnv_genes))



    subnet = net[net.endg.isin(variant_genes)]

    subnet = subnet[subnet.startg.isin(outlier_genes)]

    subnet = subnet[subnet.startg != subnet.endg]

    df.at[sub, 'avg_shortest_distance'] = subnet.num_connector_genes.mean()

SG001
SG002
SG003
SG006
SG007
SG011
SG021
SG022
SG023
SG024
SG025
SG026
SG027
SG030
SG031
SG069
SG155
SG037
SG038
SG039
SG040
SG041
SG042
SG043
SG044
SG045
SG046
SG148
SG149
SG150
SG151
SG152


In [39]:
df.to_csv('avg_dist.tsv', sep='\t', index=False)

In [40]:
df.to_csv('avg_dist.tsv', sep='\t')